In [118]:
import pandas as pd
from sqlalchemy import create_engine, text

In [119]:

DB_USER = "myuser"
DB_PASSWORD = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydatabase"

DATABASE_URL = (
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}"
    f"@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

engine = create_engine(DATABASE_URL)
print("✅ Connected to PostgreSQL")

✅ Connected to PostgreSQL


In [120]:
from sqlalchemy import text

def create_tables():
    queries = [
        # 1. Branches (Master)
        """
        CREATE TABLE IF NOT EXISTS branches (
            branch_no INT PRIMARY KEY
        );
        """,

        # 2. Gender (Lookup)
        """
        CREATE TABLE IF NOT EXISTS gender (
            gender_no SERIAL PRIMARY KEY,
            gender_name VARCHAR(50) UNIQUE NOT NULL
        );
        """,

        # 3. Careers (Lookup)
        """
        CREATE TABLE IF NOT EXISTS careers (
            career_no SERIAL PRIMARY KEY,
            career_name VARCHAR(200) UNIQUE NOT NULL
        );
        """,

        # 4. Members (Main Table - เก็บเฉพาะข้อมูลพื้นฐาน)
        """
        CREATE TABLE IF NOT EXISTS members (
            member_id INT PRIMARY KEY,
            gender_no INT NOT NULL REFERENCES gender(gender_no),
            first_name VARCHAR(150) NOT NULL,
            last_name VARCHAR(150) NOT NULL,
            branch_no INT NOT NULL REFERENCES branches(branch_no)
        );
        """,

        # 5. Birthdays
        """
        CREATE TABLE IF NOT EXISTS birthdays (
            member_id INT PRIMARY KEY REFERENCES members(member_id),
            birthday DATE NOT NULL,
            CHECK (birthday <= CURRENT_DATE)
        );
        """,

        # 6. Registration
        """
        CREATE TABLE IF NOT EXISTS registration (
            member_id INT PRIMARY KEY REFERENCES members(member_id),
            registration_date DATE NOT NULL,
            CHECK (registration_date <= CURRENT_DATE)
        );
        """,

        # 7. Approval
        """
        CREATE TABLE IF NOT EXISTS approval (
            member_id INT PRIMARY KEY REFERENCES members(member_id),
            approval_date DATE
        );
        """,

        # 8. Income
        """
        CREATE TABLE IF NOT EXISTS income (
            member_id INT PRIMARY KEY REFERENCES members(member_id),
            career_no INT REFERENCES careers(career_no),
            income NUMERIC(15,2) CHECK (income >= 0)
        );
        """,

        # 9. Addresses
        """
        CREATE TABLE IF NOT EXISTS addresses (
            member_id INT PRIMARY KEY REFERENCES members(member_id),
            house_no VARCHAR(100),
            village_no VARCHAR(50),
            road VARCHAR(255),
            sub_area VARCHAR(150),
            district_area VARCHAR(150),
            province VARCHAR(150),
            postal_code VARCHAR(10),
            CHECK (postal_code IS NULL OR postal_code ~ '^[0-9]{5}$')
        );
        """
    ]

    with engine.begin() as conn:
        for q in queries:
            conn.execute(text(q))

    print("✅ สร้างตารางครบทั้ง 9 ตารางเรียบร้อยแล้ว")

if __name__ == "__main__":
    create_tables()

✅ สร้างตารางครบทั้ง 9 ตารางเรียบร้อยแล้ว


In [122]:
pd.read_sql(
    "SELECT * FROM public.members;",
    engine
)

,member_id,gender_no,first_name,last_name,branch_no
0,500001,1,ชัชวาล,เลิศพิทักษ์,3
1,500002,1,อารีฟีน,ฐานวัฒน์,4
2,500003,2,ซาฟียะห์,สาระสิทธิ์,4
3,500004,1,กนกวรรณ,กาญจนพงศ์,4
4,500005,3,ศิริลักษณ์,ศานติธรรม,2
...,...,...,...,...,...
295,500296,3,เอกลักษณ์,สุวรรณดี,3
296,500297,3,ซาบีนะห์,อารีย์สกุล,2
297,500298,1,เจตน์,มหาดำรงค์,2
298,500299,2,จิรายุ,เกรียงไกรกล้า,2
